In [6]:
from phi.agent import Agent
from phi.model.groq import Groq

# Directly include the API key
API_KEY = "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"  # Replace with your actual API key

agent = Agent(
    model=Groq(id="llama-3.2-90b-vision-preview", api_key=API_KEY)
)
agent.print_response("Share a 2-sentence love story between dosa and samosa")


ImportError: cannot import name 'identity' from 'phi.utils' (c:\Users\tusha\AppData\Local\Programs\Python\Python312\Lib\site-packages\phi\utils\__init__.py)

In [3]:
pip install phi --upgrade --force-reinstall


  Using cached phi-0.6.7-py3-none-any.whl
  Attempting uninstall: phi
    Found existing installation: phi 0.6.7
    Uninstalling phi-0.6.7:
      Successfully uninstalled phi-0.6.7
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install phi --upgrade

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_groq import ChatGroq
import os
# from dotenv import load_dotenv

# load_dotenv()
# llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.2-90b-text-preview")
# Directly specifying the API key in the code without using environment variables
llm = ChatGroq(groq_api_key="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", model_name="llama-3.3-70b-versatile")



if __name__ == "__main__":
    response = llm.invoke("Two most important ingradient in samosa are ")
    print(response.content)

The two most important ingredients in a traditional samosa are:

1. **Potatoes**: Boiled, mashed, and spiced potatoes are a key filling ingredient in most samosa recipes.
2. **All-purpose flour (or Maida)**: This is used to make the dough that wraps around the filling, giving samosas their distinctive shape and crispy texture.

Of course, other ingredients like onions, peas, spices, and oil are also important, but potatoes and all-purpose flour are the fundamental components of a classic samosa.


In [13]:
from langchain_groq import ChatGroq
from phi.data import phiData
import os

# Load Groq API Key (if you prefer environment variables)
# from dotenv import load_dotenv
# load_dotenv()
# llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.2-90b-text-preview")

# Directly specify the API key and model name
llm = ChatGroq(groq_api_key="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", model_name="llama-3.3-70b-versatile")

# Example of using phiData to load some structured data
def load_samosa_data():
    # Let's assume phiData is used to load ingredients data or any other structured data
    # Here, we're just creating a small dataset for the sake of an example
    data = {
        "samosa": {
            "ingredients": ["potatoes", "spices", "peas", "flour", "oil"],
            "description": "A popular snack, made from a crispy pastry filled with a savory mixture."
        }
    }
    
    # Using phiData to handle structured data (could be a dataframe or other form)
    samosa_data = phiData(data)
    
    return samosa_data

# Using phiData in processing (here, we print the structured data)
samosa_data = load_samosa_data()
print(samosa_data)

if __name__ == "__main__":
    # Call the Groq model using the input from phiData or other sources
    response = llm.invoke("Two most important ingredients in samosa are")
    print(response.content)


ImportError: cannot import name 'identity' from 'phi.utils' (c:\Users\tusha\AppData\Local\Programs\Python\Python312\Lib\site-packages\phi\utils\__init__.py)

In [16]:
pip install yfinance

In [18]:
import os
from typing import Dict, Any
import yfinance as yf
from langchain_core.tools import BaseTool
from langchain_core.language_models import BaseLanguageModel
from langchain.agents import AgentExecutor, create_react_agent
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def get_company_symbol(company: str) -> str:
    """Get the stock symbol for a given company."""
    symbols = {
        "Phidata": "MSFT",
        "Infosys": "INFY",
        "Tesla": "TSLA",
        "Apple": "AAPL",
        "Microsoft": "MSFT",
        "Amazon": "AMZN",
        "Google": "GOOGL",
    }
    return symbols.get(company, "Unknown")

class YFinanceStockTool(BaseTool):
    """Custom tool for fetching stock information using yfinance."""
    
    name = "yfinance_stock_tool"
    description = "Useful for getting stock information such as price, recommendations, and fundamentals"
    
    def __init__(self, stock_symbol: str):
        super().__init__()
        self.stock_symbol = stock_symbol

    def _run(self, query: str) -> str:
        """Run method for synchronous execution."""
        try:
            stock = yf.Ticker(self.stock_symbol)
            
            if query == "stock_price":
                # Get the most recent closing price
                history = stock.history(period="1d")
                return f"Current stock price for {self.stock_symbol}: ${history['Close'][-1]:.2f}"
            
            elif query == "analyst_recommendations":
                # Get analyst recommendations
                recommendations = stock.recommendations
                return f"Analyst Recommendations for {self.stock_symbol}:\n{recommendations.to_string()}"
            
            elif query == "stock_fundamentals":
                # Get key stock fundamentals
                info = stock.info
                fundamentals = {
                    "Company Name": info.get('longName', 'N/A'),
                    "Sector": info.get('sector', 'N/A'),
                    "Market Cap": info.get('marketCap', 'N/A'),
                    "PE Ratio": info.get('trailingPE', 'N/A'),
                    "Dividend Yield": info.get('dividendYield', 'N/A'),
                    "52 Week High": info.get('fiftyTwoWeekHigh', 'N/A'),
                    "52 Week Low": info.get('fiftyTwoWeekLow', 'N/A')
                }
                return "\n".join(f"{k}: {v}" for k, v in fundamentals.items())
            
            else:
                return "Invalid query. Use 'stock_price', 'analyst_recommendations', or 'stock_fundamentals'."
        
        except Exception as e:
            return f"Error retrieving stock information: {str(e)}"

    def _arun(self, query: str):
        """Run method for asynchronous execution."""
        raise NotImplementedError("Async method not supported")

def create_stock_agent(groq_api_key: str, model_name: str = "llama-3.1-70b-versatile"):
    """Create an agent for stock analysis."""
    # Initialize Groq Language Model
    llm = ChatGroq(temperature=0, model_name=model_name, groq_api_key=groq_api_key)
    
    # Create tools
    tools = [
        YFinanceStockTool("TSLA"),
        YFinanceStockTool("MSFT"),
        BaseTool(
            name="get_company_symbol", 
            func=get_company_symbol,
            description="Get stock symbol for a given company"
        )
    ]
    
    # Define a custom prompt template
    template = """You are a professional financial analyst. Use the following tools to analyze stocks:

    1. Fetch stock prices
    2. Get analyst recommendations
    3. Retrieve stock fundamentals

    Question: {input}
    
    {agent_scratchpad}"""

    prompt = PromptTemplate(
        template=template,
        input_variables=["input", "agent_scratchpad"]
    )

    # Create the agent
    agent = create_react_agent(
        llm=llm, 
        tools=tools, 
        prompt=prompt
    )

    # Create the agent executor
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        verbose=True,
        handle_parsing_errors=True
    )

    return agent_executor

def main():
    # Replace with your actual Groq API key
    groq_api_key = os.getenv("GROQ_API_KEY", "your_groq_api_key_here")
    
    # Create the stock analysis agent
    stock_agent = create_stock_agent(groq_api_key)
    
    # Run the agent
    response = stock_agent.run(
        "Summarize and compare analyst recommendations and fundamentals for TSLA and MSFT. Show key insights."
    )
    print(response)

if __name__ == "__main__":
    main()

PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.9/u/model-field-overridden

In [22]:
import os
from typing import Dict, Any, Optional
import yfinance as yf
from langchain_core.tools import BaseTool
from langchain_core.language_models import BaseLanguageModel
from langchain.agents import AgentExecutor, create_react_agent
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

def get_company_symbol(company: str) -> str:
    """Get the stock symbol for a given company."""
    symbols = {
        "Phidata": "MSFT",
        "Infosys": "INFY",
        "Tesla": "TSLA",
        "Apple": "AAPL",
        "Microsoft": "MSFT",
        "Amazon": "AMZN",
        "Google": "GOOGL",
    }
    return symbols.get(company, "Unknown")

class YFinanceStockToolInput(BaseModel):
    """Input model for YFinanceStockTool."""
    stock_symbol: str = Field(description="Stock symbol to analyze")
    query: str = Field(description="Type of query to run", default="stock_fundamentals")

class YFinanceStockTool(BaseTool):
    """Custom tool for fetching stock information using yfinance."""
    
    name: str = "yfinance_stock_tool"
    description: str = "Useful for getting stock information such as price, recommendations, and fundamentals"
    args_schema: type[BaseModel] = YFinanceStockToolInput

    def _run(self, stock_symbol: str, query: str = "stock_fundamentals") -> str:
        """Run method for synchronous execution."""
        try:
            stock = yf.Ticker(stock_symbol)
            
            if query == "stock_price":
                # Get the most recent closing price
                history = stock.history(period="1d")
                return f"Current stock price for {stock_symbol}: ${history['Close'][-1]:.2f}"
            
            elif query == "analyst_recommendations":
                # Get analyst recommendations
                recommendations = stock.recommendations
                return f"Analyst Recommendations for {stock_symbol}:\n{recommendations.to_string()}"
            
            elif query == "stock_fundamentals":
                # Get key stock fundamentals
                info = stock.info
                fundamentals = {
                    "Company Name": info.get('longName', 'N/A'),
                    "Sector": info.get('sector', 'N/A'),
                    "Market Cap": info.get('marketCap', 'N/A'),
                    "PE Ratio": info.get('trailingPE', 'N/A'),
                    "Dividend Yield": info.get('dividendYield', 'N/A'),
                    "52 Week High": info.get('fiftyTwoWeekHigh', 'N/A'),
                    "52 Week Low": info.get('fiftyTwoWeekLow', 'N/A')
                }
                return "\n".join(f"{k}: {v}" for k, v in fundamentals.items())
            
            else:
                return "Invalid query. Use 'stock_price', 'analyst_recommendations', or 'stock_fundamentals'."
        
        except Exception as e:
            return f"Error retrieving stock information: {str(e)}"

def create_stock_agent(groq_api_key: str, model_name: str = "llama-3.1-70b-versatile"):
    """Create an agent for stock analysis."""
    # Initialize Groq Language Model
    llm = ChatGroq(temperature=0, model_name=model_name, groq_api_key=groq_api_key)
    
    # Create tools
    tools = [
        YFinanceStockTool(),
        BaseTool(
            name="get_company_symbol", 
            description="Get stock symbol for a given company",
            func=get_company_symbol,
        )
    ]
    
    # Define a custom prompt template
    template = """You are a professional financial analyst. Use the following tools to analyze stocks:

    1. Fetch stock prices
    2. Get analyst recommendations
    3. Retrieve stock fundamentals

    Tip: Use the yfinance_stock_tool with a specific stock symbol and query type.

    Question: {input}
    
    {agent_scratchpad}"""

    prompt = PromptTemplate(
        template=template,
        input_variables=["input", "agent_scratchpad"]
    )

    # Create the agent
    agent = create_react_agent(
        llm=llm, 
        tools=tools, 
        prompt=prompt
    )

    # Create the agent executor
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        verbose=True,
        handle_parsing_errors=True
    )

    return agent_executor

def main():
    # Replace with your actual Groq API key
    groq_api_key = os.getenv("GROQ_API_KEY", "your_groq_api_key_here")
    
    # Create the stock analysis agent
    stock_agent = create_stock_agent(groq_api_key)
    
    # Run the agent
    response = stock_agent.run(
        "Summarize and compare analyst recommendations and fundamentals for TSLA and MSFT. Show key insights."
    )
    print(response)

if __name__ == "__main__":
    main()

TypeError: Can't instantiate abstract class BaseTool without an implementation for abstract method '_run'

In [21]:
pip install yfinance langchain-core langchain-groq python-dotenv pydantic

Note: you may need to restart the kernel to use updated packages.
